In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import pickle
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential 
from keras import layers
from keras import regularizers
from keras import backend as K
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
import csv

In [26]:
from keras.preprocessing.text import Tokenizer

In [2]:
data = pd.read_csv("data.csv", encoding = "ISO-8859-1", engine="python")
data.columns = ["label", "time", "date", "query", "username", "text"]

In [3]:
data.head()

,label,time,date,query,username,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [4]:
data.tail()

,label,time,date,query,username,text
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [113]:
# The top results have the positive results while the bottom results have negative results

In [5]:
data.columns

Index(['label', 'time', 'date', 'query', 'username', 'text'], dtype='object')

In [6]:
print('lenght of data is', len(data))

lenght of data is 1599999


In [116]:
# The overall dataset is quite large. Therefore a portion of this data shall be used.

In [7]:
data.shape

(1599999, 6)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   label     1599999 non-null  int64 
 1   time      1599999 non-null  int64 
 2   date      1599999 non-null  object
 3   query     1599999 non-null  object
 4   username  1599999 non-null  object
 5   text      1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [9]:
np.sum(data.isnull().any(axis=1))

0

In [120]:
# There are no null values. Therefore, missing value treatment is not needed.

In [10]:
# We only need the text and the label column. Therefore only those 2 columns are selected

data=data[['text','label']]

In [11]:
# Values for label are 0 and 4. The 4 values are changed to 1

data['label'][data['label']==4]=1

In [12]:
# We seperate out the positive and negative samples and select 20000 from each. 
# The original dataset is too big. Therefore, having a smaller sample helps in increasing training speed
# The original dataset is also imbalanced. Therefore, taking 20000 values from both hepls make the dataset balanced.
# The number of samples can be increased to improve accuracy.

data_pos = data[data['label'] == 1]
data_neg = data[data['label'] == 0]

data_pos = data_pos.iloc[:int(30000)]
data_neg = data_neg.iloc[:int(30000)]

# Merging the data

data = pd.concat([data_pos, data_neg])

In [13]:
# We convert the text to lowercase

data['text']=data['text'].str.lower()

In [14]:
data.tail()

,text,label
29995,oh ffs! i've been here all fucking day. why de...,0
29996,leaving britney-just found out jon from new ki...,0
29997,"@neomic i havta' go pee, but im scared to walk...",0
29998,nooooooooooooooo!!!!!! school today. but the w...,0
29999,back to school tomorow,0


In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omraw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [18]:
# The data needs to be cleaned. 

stopwords_list = stopwords.words('english')

", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [19]:
STOPWORDS = set(stopwords.words('english'))
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

data['text'] = data['text'].apply(lambda text: cleaning_stopwords(text))
#data['text'].head()



english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

data['text']= data['text'].apply(lambda x: cleaning_punctuations(x))
#data['text'].tail()



def cleaning_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

data['text'] = data['text'].apply(lambda x: cleaning_repeating_char(x))
#data['text'].tail()



def cleaning_email(data):
    return re.sub('@[^\s]+', ' ', data)

data['text']= data['text'].apply(lambda x: cleaning_email(x))
#data['text'].tail()



def cleaning_URLs(data):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',data)

data['text'] = data['text'].apply(lambda x: cleaning_URLs(x))
#data['text'].tail()



def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)

data['text'] = data['text'].apply(lambda x: cleaning_numbers(x))
#data['text'].tail()

In [20]:
data.head()

,text,label
799999,love healthuandpets u guys r best,1
800000,im meting one besties tonight cant wait girl talk,1
800001,darealsunisakim thanks twiter ad sunisa got me...,1
800002,sick realy cheap hurts much eat real fod plus ...,1
800003,lovesbroklyn efect everyone,1


In [21]:
# Tokenization

tokenizer = RegexpTokenizer(r'\w+')
data['text'] = data['text'].apply(tokenizer.tokenize)

data['text'].head()

799999             [love, healthuandpets, u, guys, r, best]
800000    [im, meting, one, besties, tonight, cant, wait...
800001    [darealsunisakim, thanks, twiter, ad, sunisa, ...
800002    [sick, realy, cheap, hurts, much, eat, real, f...
800003                      [lovesbroklyn, efect, everyone]
Name: text, dtype: object

In [22]:
# Stemming

st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data

data['text']= data['text'].apply(lambda x: stemming_on_text(x))

data.head()

,text,label
799999,"[love, healthuandpets, u, guys, r, best]",1
800000,"[im, meting, one, besties, tonight, cant, wait...",1
800001,"[darealsunisakim, thanks, twiter, ad, sunisa, ...",1
800002,"[sick, realy, cheap, hurts, much, eat, real, f...",1
800003,"[lovesbroklyn, efect, everyone]",1


In [23]:
# Lemmatization

lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data

data['text'] = data['text'].apply(lambda x: lemmatizer_on_text(x))

data.head()

,text,label
799999,"[love, healthuandpets, u, guys, r, best]",1
800000,"[im, meting, one, besties, tonight, cant, wait...",1
800001,"[darealsunisakim, thanks, twiter, ad, sunisa, ...",1
800002,"[sick, realy, cheap, hurts, much, eat, real, f...",1
800003,"[lovesbroklyn, efect, everyone]",1


In [24]:
X=data['text']
y=[]

for i in range(len(data)):
  l=[0,0]
  temp=data.iloc[i,1]
  l[temp-1]=1
  y.append(l)

print(y[0])
print(y[-1])

# [1,0] for positive emotion, [0,1] for negative emotion

[1, 0]
[0, 1]


In [133]:
max_len=0
min_len=1000
sum_len=0
avg_len=0


for i in X:
    if len(i)>max_len:
        max_len=len(i)
    if len(i)< min_len:
        min_len=len(i)
    sum_len=sum_len+len(i)
avg_len=sum_len/len(X)
print(max_len)
print(min_len)
print(sum_len)
print(avg_len)
print(len(X))

39
0
466831
7.780516666666666
60000


In [27]:
max_len = 100
num_words = 2000
tok = Tokenizer(num_words=2000)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [28]:
sequences_matrix.shape

(60000, 100)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(sequences_matrix, y, test_size=0.2, random_state=2)

In [30]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [31]:
model = Sequential()
model.add(layers.Embedding(2000, 40, input_length=max_len))
model.add(layers.LSTM(64))
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(2,activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer=sgd, metrics=['accuracy'])

C:\Users\omraw\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 40)           80000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                26880     
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 164,802
Trainable params: 164,802
Non-trainable params: 0
__________________________________________________

In [33]:
history=model.fit(X_train, y_train, epochs=40,validation_data=(X_test, y_test))

Epoch 1/40
1500/1500 [==============================] - 39s 25ms/step - loss: 0.6926 - accuracy: 0.5122 - val_loss: 0.6912 - val_accuracy: 0.5140
Epoch 2/40
1500/1500 [==============================] - 43s 28ms/step - loss: 0.6612 - accuracy: 0.5911 - val_loss: 0.5589 - val_accuracy: 0.7143
Epoch 3/40
1500/1500 [==============================] - 41s 28ms/step - loss: 0.5433 - accuracy: 0.7244 - val_loss: 0.5298 - val_accuracy: 0.7378
Epoch 4/40
1500/1500 [==============================] - 42s 28ms/step - loss: 0.5093 - accuracy: 0.7519 - val_loss: 0.5332 - val_accuracy: 0.7268
Epoch 5/40
1500/1500 [==============================] - 42s 28ms/step - loss: 0.4977 - accuracy: 0.7595 - val_loss: 0.5057 - val_accuracy: 0.7533
Epoch 6/40
1500/1500 [==============================] - 44s 29ms/step - loss: 0.4915 - accuracy: 0.7641 - val_loss: 0.5137 - val_accuracy: 0.7462
Epoch 7/40
1500/1500 [==============================] - 45s 30ms/step - loss: 0.4928 - accuracy: 0.7641 - val_loss: 0.5244 -

KeyboardInterrupt: 

In [34]:
list = ['This is the best day of my life', 'I feel like dying','I want to die I am a bad person','I feel like killing myself','I am happy','I am sad','I am very happy']

sequences = tok.texts_to_sequences(list)
testing_sequences = sequence.pad_sequences(sequences,maxlen=max_len)

y_samples = np.array(testing_sequences)
model.predict(y_samples)

array([[0.9653226 , 0.03467736],
       [0.0217372 , 0.9782628 ],
       [0.08798612, 0.9120139 ],
       [0.74056214, 0.25943792],
       [0.8720739 , 0.12792613],
       [0.00138258, 0.9986174 ],
       [0.85858035, 0.14141971]], dtype=float32)

In [35]:
y_samples

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,  390,  355,  861,   93,    3,  315, 1345,
         106],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    

In [36]:
print(list)
print(model.predict(y_samples))

['This is the best day of my life', 'I feel like dying', 'I want to die I am a bad person', 'I feel like killing myself', 'I am happy', 'I am sad', 'I am very happy']
[[0.9653226  0.03467736]
 [0.0217372  0.9782628 ]
 [0.08798612 0.9120139 ]
 [0.74056214 0.25943792]
 [0.8720739  0.12792613]
 [0.00138258 0.9986174 ]
 [0.85858035 0.14141971]]


In [154]:
import pickle

model.save('model.h5', history) 